In [ ]:
import cv2
import json, os
import glob
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world
from aquabyte.visualize import Visualizer
import random
from scipy.stats import norm
from urllib.parse import urlparse
from PIL import Image, ImageDraw, ImageFont
from multiprocessing import Pool

pd.set_option('display.max_rows', 500)

<h1> Select the date, site_id, and pen_id for which we would like the stitched video </h1>

In [ ]:

rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from lati_fish_detections_lice_annotations
    where pen_id=37 and captured_at >= '2019-09-19' and captured_at < '2019-09-20';
"""
df = rds_access_utils.extract_from_database(query)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
def get_bucket_key(url):
    parsed_url = urlparse(url, allow_fragments=False)
    if parsed_url.netloc.startswith('s3'):
        url_components = parsed_url.path.lstrip('/').split('/')
        bucket, key = url_components[0], os.path.join(*url_components[1:])
    else:
        bucket = parsed_url.netloc.split('.')[0]
        key = parsed_url.path.lstrip('/')
    return bucket, key

start_date, end_date = '2019-09-19', '2019-09-20'
image_url = df.image_url.iloc[0]
bucket, key = get_bucket_key(image_url)
inbound_bucket = 'aquabyte-frames-resized-inbound'
s3_folder = key[:key.index(start_date) + len(start_date)]

In [ ]:
generator = s3_access_utils.get_matching_s3_keys(inbound_bucket, s3_folder, suffixes=['capture.json'])

In [ ]:
keys = []
for key in generator:
    keys.append(key)

In [ ]:
s3_key_dirs = sorted(list(set([os.path.dirname(f) for f in keys])))

In [ ]:
x_padding = 50 * (512 / 4096.0)
y_padding = 50 * (512 / 3000.0)
def process_s3_key_dir(s3_key_dir):
    try:
        left_f = s3_access_utils.download_from_s3(inbound_bucket, os.path.join(s3_key_dir, 'left_frame.resize_512_512.jpg'))
        right_f = s3_access_utils.download_from_s3(inbound_bucket, os.path.join(s3_key_dir, 'right_frame.resize_512_512.jpg'))
        crop_metadata_f = s3_access_utils.download_from_s3(inbound_bucket, os.path.join(s3_key_dir, 'crops.json'))

        # open images and metadata files
        left_im = Image.open(left_f)
        right_im = Image.open(right_f)
        crop_metadata = json.load(open(crop_metadata_f))

        # draw boxes on images
        left_draw = ImageDraw.Draw(left_im)
        right_draw = ImageDraw.Draw(right_im)
        anns = crop_metadata['annotations']
        for ann in anns:
            if ann['image_id'] == 1:
                left_draw.rectangle([(ann['bbox'][1] - x_padding, 
                                      ann['bbox'][0] - y_padding), 
                                     (ann['bbox'][3] + x_padding, 
                                      ann['bbox'][2] + y_padding)])
            elif ann['image_id'] == 2:
                right_draw.rectangle([(ann['bbox'][1], ann['bbox'][0]), (ann['bbox'][3], ann['bbox'][2])])

        # stitch images
        result = Image.new('RGB', (512*2, 512))
        result.paste(im=left_im, box=(0, 0))
        result.paste(im=right_im, box=(512, 0))

        # write timestamp on stitched image
        result_draw = ImageDraw.Draw(result)
        ts = [c for c in left_f.split('/') if c.startswith('at=')][0]
        result_draw.text((0, 0), ts, (255, 255, 255))

        output_f = left_f.replace(root_dir, output_base_dir).replace('left_', 'stereo_')
        if not os.path.exists(os.path.dirname(output_f)):
            os.makedirs(os.path.dirname(output_f))
        result.save(output_f)
    except Exception as e:
        print(e)



In [ ]:
root_dir = '/root/data/s3'
output_base_dir = '/root/data/alok/biomass_estimation/playground/bremnes_tittelsnes_stitched_results'
pool = Pool(20)
pool.map(process_s3_key_dir, s3_key_dirs)
    

In [ ]:
output_base_dir

In [ ]:
image_fs = sorted(glob.glob(os.path.join(output_base_dir, '**', '*.jpg'), recursive=True))

In [ ]:
import cv2
import numpy as np

im = cv2.imread(image_fs[0])
height,width,layers = im.shape
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video=cv2.VideoWriter('/root/data/alok/2019-09-19_bremnes_tittelsnes_video.avi', fourcc, 4, (width,height), True)
for idx, image_f in enumerate(image_fs):
    if idx % 1000 == 0:
        print(idx)
    im = cv2.imread(image_f, cv2.IMREAD_COLOR)
    video.write(im)

cv2.destroyAllWindows()
video.release()